<a href="https://colab.research.google.com/github/dkhos17/Kaggle-NLP/blob/main/dkhos17_nlp_hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
QUESTION1, QUESTION2, IS_DUPLICATE = ["How do I flow traffic to my website?"], ["How do I build traffic to my website?"], [1]
QUESTION1, QUESTION2, IS_DUPLICATE = ["How do I flow traffic to my website?"], ["How do I build home near street?"], [0]

In [ ]:
MAX_QUESTION_LEN = 30

def configure_data(data):
    def config(q, dels=False):
        q = q.replace("'s", " is")
        q = q.replace("'re", " are")
        q = q.replace("'m", " am")
        q = q.replace("'ll", " will")
        q = q.replace("'d", " would")
        q = q.replace("'t", " not")
        return ' '.join(q.split(' ')[:MAX_QUESTION_LEN])
    
    data['question1'] = data['question1'].apply(lambda q : config(q)) 
    data['question2'] = data['question2'].apply(lambda q : config(q))
    return data

my_data = pd.DataFrame(list(zip(QUESTION1, QUESTION2, IS_DUPLICATE)), columns =['question1', 'question2', 'is_duplicate'])
my_data = configure_data(my_data)
my_data.head()

In [ ]:
!pip install transformers

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import BertModel, AdamW, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

In [ ]:
class Dataset():
    def __init__(self, questions, targets=None):
        self.questions = questions
        self.targets = targets

    def __len__(self):
        return len(self.questions[0])

    def __getitem__(self, index):
        # [CLS] Q1 [SEP] Q2 [SEP] 0 0 0 ... LEN = (2*MAX_QUESTION_LEN+5)
        ei = tokenizer(self.questions[0][index], self.questions[1][index], add_special_tokens=True, return_token_type_ids=True, 
                       return_attention_mask=True, padding='max_length', max_length = (2*MAX_QUESTION_LEN+5), truncation=True)
        ita = (torch.LongTensor(ei['input_ids']), torch.LongTensor(ei['token_type_ids']), torch.LongTensor(ei['attention_mask']))
        
        return ita, self.targets[index] if self.targets else 0.0

In [ ]:
MyDataSet = Dataset([QUESTION1, QUESTION2], IS_DUPLICATE)
MyDataLoader = DataLoader(MyDataSet, batch_size=len(IS_DUPLICATE))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
class BERT(nn.Module):
    def __init__(self, drop=0.15, hdim=768):
        super().__init__()
        self.pretrained = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(drop)
        self.linear = nn.Linear(hdim,1)

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
        outputs = self.pretrained(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        outputs = self.dropout(outputs[1])
        return self.linear(outputs)

model = BERT().to(device)
model.load_state_dict(torch.load('gdrive/My Drive/Colab Notebooks/MyBertModel'))

In [ ]:
model.eval()
PREDICTION = []
with torch.no_grad():
    for b, batch in enumerate(MyDataLoader):
        # get current batch from data
        Q_batch, _ = batch
        Q_input_ids, Q_token_type_ids, Q_attention_mask= Q_batch
        
        # predict targets for current batch and learn by comparing it to real targets with loss func.
        Q_pred = model(input_ids=Q_input_ids.to(device), token_type_ids=Q_token_type_ids.to(device), attention_mask=Q_attention_mask.to(device)).to(device)

        # predict batch according to our trained model
        PREDICTION = torch.sigmoid(Q_pred).cpu().numpy().squeeze()

In [ ]:
# save data to submit
result = pd.DataFrame({"is_duplicate": IS_DUPLICATE, "my_prediction" : PREDICTION})
result.to_csv("result.csv", index=False)

In [ ]:
result.head()